In [1]:
import scraping_class
import random
import time
import tqdm
import requests
import pandas as pd
from bs4 import BeautifulSoup

logfile = 'log.csv'## name your log file.
connector = scraping_class.Connector(logfile)

# First we want a link for each result page in careergate

def link_page(n): # defining a function that creates a link for a given page number
    link =  f'https://careergate.cbs.dk/da/jobs?remote=1&limit=10&offset={n}'
    return link

links = [link_page(n) for n in range(0,320,10)] #storing links for all the pages


In [35]:
# Now we want a list of all the job links in each page 

job_links = []
for link in links : 
    response,call_id = connector.get(link,'searchpage')
    soup = BeautifulSoup(response.text, "html5lib") # get html for each page
    job_link = soup.find_all('a', class_="job-box") # find all job-box objects
    job_links += job_link

    
jobs = [job_link.get('href') for job_link in job_links] # get all the job links in one list 
job_links_total = list(dict.fromkeys(jobs)) # remove duplicates
len(job_links_total) # make sure this matches with the total from the search page


In [45]:
# add the 'https://careergate.cbs.dk' to the links so that they can be used later 
url_prefix = 'https://careergate.cbs.dk'
job_links_total = [url_prefix + job_link for job_link in job_links_total]

In [363]:
from tqdm import tqdm_notebook as tqdm

job_data = [] # this will eventually be our dataframe

for link in job_links_total:
    response,call_id = connector.get(link,'job posting')
    soup = BeautifulSoup(response.text, "lxml") # get html for each page
    descrip = soup.find('article', class_ = "box-item job-content")#.get_text()
    box = soup.find('div', class_ = "content-description").get_text()
    box = box.replace("\n", " ")
    box = " ".join(box.split())
    location = re.findall(r'(?<=Sted)(.*)(?=Fjernarbejde)', box)
    category = re.findall(r'(?<=Kategori)(.*)(?=Type)', box)
    type_ = re.findall(r'(?<=Type)(.*)(?=Kompetencer)', box)
    skills = re.findall(r'(?<=Kompetencer)(.*)(?=Must-have sprog)', box)
    languages = re.findall(r'(?<=Must-have sprog)(.*)', box)
    description = re.findall(r'(?<=<p>)(.*)(?=</p>)', str(descrip))
    job = [str(location), str(category), str(type_), str(skills), str(languages), str(description)]
    job_data.append(job)


In [401]:
#transforming into dataframe
import pandas as pd
df = pd.DataFrame(job_data)
df.columns = ["Location", "Category", "Type", "Skills", "Language", "Description"]

# code to clean the description column from html 
df['Description'] = [BeautifulSoup(text).get_text() for text in df['Description'] ]

# taking all values out of the brackets
columns = ["Location", "Category", "Type", "Skills", "Language", "Description"]
for column in columns:
    df[column] = df[column].str.strip('[]').astype(str)
    df[column] = df[column].str.strip(' "" ')
    df[column] = df[column].str.strip(' '' ')

In [375]:
print(df)